In [76]:
import tensorflow as tf
import numpy as np 
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle

RANDOM_SEED = 42

In [77]:
dataset_path = 'model/keypoint_classifier/data/keypoint.csv'
model_save_path = 'model/keypoint_classifier/models/keypoint_classifier.hdf5'

In [78]:
encoder = LabelEncoder()
X_dataset = np.loadtxt(dataset_path, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset_path, delimiter=',', dtype='int32', usecols=(0))
y_dataset_encoded = encoder.fit_transform(y_dataset)
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset_encoded, train_size=0.75, random_state=RANDOM_SEED)
with open("le.obj","wb") as f:
    pickle.dump(encoder,f)
NUM_CLASSES = len(encoder.classes_)

In [79]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [80]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_12 (Dropout)        (None, 42)                0         
                                                                 
 dense_18 (Dense)            (None, 20)                860       
                                                                 
 dropout_13 (Dropout)        (None, 20)                0         
                                                                 
 dense_19 (Dense)            (None, 10)                210       
                                                                 
 dense_20 (Dense)            (None, 3)                 33        
                                                                 
Total params: 1,103
Trainable params: 1,103
Non-trainable params: 0
_________________________________________________________________


In [81]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=0, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [82]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [83]:
model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/10
19/19 [==============================] - 0s 10ms/step - loss: 1.1241 - accuracy: 0.3392 - val_loss: 1.0754 - val_accuracy: 0.4040
Epoch 2/10
19/19 [==============================] - 0s 4ms/step - loss: 1.0737 - accuracy: 0.3789 - val_loss: 1.0264 - val_accuracy: 0.3813
Epoch 3/10
19/19 [==============================] - 0s 4ms/step - loss: 1.0426 - accuracy: 0.4312 - val_loss: 0.9840 - val_accuracy: 0.4495
Epoch 4/10
19/19 [==============================] - 0s 4ms/step - loss: 1.0045 - accuracy: 0.4675 - val_loss: 0.9402 - val_accuracy: 0.5530
Epoch 5/10
19/19 [==============================] - 0s 4ms/step - loss: 0.9764 - accuracy: 0.4869 - val_loss: 0.8931 - val_accuracy: 0.6995
Epoch 6/10
19/19 [==============================] - 0s 4ms/step - loss: 0.9130 - accuracy: 0.5603 - val_loss: 0.8316 - val_accuracy: 0.7247
Epoch 7/10
19/19 [==============================] - 0s 4ms/step - loss: 0.8845 - accuracy: 0.5831 - val_loss: 0.7689 - val_accuracy: 0.8384
Epoch 8/10
19/19 [=

In [84]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

4/4 [==============================] - 0s 997us/step - loss: 0.5681 - accuracy: 0.8990


In [85]:
model = tf.keras.models.load_model(model_save_path)

In [87]:
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))
print(y_test[0])

1/1 [==============================] - 0s 18ms/step
[0.03313294 0.38344193 0.58342516]
2
2
